In [ ]:
"""
Script: prefilter_comparison_constant_velocity.py

Purpose
-------
Compare three prefilter configurations (Pleated, Electrostatic, Combo) on the same figure:
  1) Prefilter ΔP (dp2) vs time (experimental)
  2) Velocity vs time (experimental)
  3) Gross ΔP normalized to a constant reference velocity v_ref (gross = dp1 + dp2)

What this script does
---------------------
- Loads three CSV blocks (paste into raw_pleated/raw_electrostatic/raw_combo).
- Smooths dp2 and velocity using a centered rolling mean (optional).
- Computes gross ΔP = dp1 + dp2, then normalizes to constant velocity:
    gross_const(t) = media(t) + K * v_ref^n
  where K is fit on the early part of the run (EARLY_FRACTION) assuming minimal loading.
- Produces a 1×3 comparison plot and saves PNG + SVG to ./fig_exports

CSV format (required columns)
-----------------------------
time,dp1,dp2,velocity
"""

from __future__ import annotations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
from pathlib import Path

# =============================
# USER SETTINGS
# =============================
V_REF_USER = 0.75          # reference velocity for normalization (m/s)
N_EXPONENT = 1.0           # exponent n in ΔP ~ v^n
ROLL_WIN = 3               # smoothing window for signals (>=2 smooths; 0/1 disables)
MIN_VEL = 1e-6             # avoid v=0 issues
LINE_WIDTH = 1
DPI = 300

# Gross-only normalization fit
EARLY_FRACTION = 0.10      # fraction of earliest points used to fit K_gross
CLIP_NEGATIVES = True      # clip inferred media term to >=0

# Visualization controls
Y_CAP_GROSS_CONST = 300.0  # Pa cap for gross-const plot (set None to disable)
SHADE_ABOVE_CAP = True
CAP_LINE_STYLE = {"ls": "--", "alpha": 0.7}

# Colors (colorblind-friendly)
C_PLEATED = "#1f77b4"
C_ESTATIC = "#ff7f0e"
C_COMBO = "#2ca02c"

# Output folder
OUTDIR = Path("fig_exports")
OUTDIR.mkdir(parents=True, exist_ok=True)

# =============================
# DATA INPUTS (paste CSVs)
# =============================
raw_pleated = """time,dp1,dp2,velocity
""".strip()

raw_electrostatic = """time,dp1,dp2,velocity
""".strip()

raw_combo = """time,dp1,dp2,velocity
""".strip()


# =============================
# Helpers
# =============================
REQUIRED_COLS = ["time", "dp1", "dp2", "velocity"]


def smooth(x: np.ndarray, win: int) -> np.ndarray:
    """Centered rolling-mean smoothing."""
    x = np.asarray(x, dtype=float)
    if not win or win < 2:
        return x
    return pd.Series(x).rolling(win, center=True, min_periods=1).mean().to_numpy()


def load_df(raw_csv: str) -> pd.DataFrame:
    """Parse a raw CSV string into a cleaned numeric DataFrame."""
    if not raw_csv.strip():
        raise ValueError("One of the datasets is empty. Paste CSV data into the raw_* string.")

    df = pd.read_csv(StringIO(raw_csv), comment="#")

    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    df = (
        df[REQUIRED_COLS]
        .apply(pd.to_numeric, errors="coerce")
        .dropna()
        .sort_values("time")
        .reset_index(drop=True)
    )

    if len(df) < 3:
        raise ValueError("Dataset has fewer than 3 valid rows after cleaning.")

    return df


def fit_K_gross(dp_gross: np.ndarray, v: np.ndarray, n: float, frac: float) -> float:
    """
    Fit K in dp_gross ≈ K * v^n using the earliest fraction of the run
    (least squares through origin).
    """
    m = max(3, int(len(v) * frac))
    v_n = np.power(v[:m], n)
    denom = np.dot(v_n, v_n)
    if denom <= 0:
        return 0.0
    K = float(np.dot(dp_gross[:m], v_n) / denom)
    return max(K, 0.0)


def gross_to_const_velocity(
    dp_gross: np.ndarray,
    v: np.ndarray,
    v_ref: float,
    n: float,
    K: float,
    clip_negatives: bool = True,
) -> np.ndarray:
    """
    Normalize gross ΔP to constant face velocity v_ref.

    dp_gross = media + K*v^n  =>  media = dp_gross - K*v^n
    gross_const(v_ref) = media + K*v_ref^n
    """
    media = dp_gross - K * np.power(v, n)
    if clip_negatives:
        media = np.maximum(media, 0.0)
    return media + K * (v_ref ** n)


def series_from_df(df: pd.DataFrame, v_ref: float, n: float) -> dict[str, np.ndarray | float]:
    """Extract and compute smoothed series + gross constant-velocity normalization."""
    t = df["time"].to_numpy(dtype=float)
    v = np.maximum(df["velocity"].to_numpy(dtype=float), MIN_VEL)
    dp1 = df["dp1"].to_numpy(dtype=float)
    dp2 = df["dp2"].to_numpy(dtype=float)

    gross_raw = dp1 + dp2
    K_gross = fit_K_gross(gross_raw, v, n=n, frac=EARLY_FRACTION)
    gross_const = gross_to_const_velocity(gross_raw, v, v_ref=v_ref, n=n, K=K_gross, clip_negatives=CLIP_NEGATIVES)

    return {
        "t": t,
        "dp2": smooth(dp2, ROLL_WIN),
        "v": smooth(v, ROLL_WIN),
        "gross_const": smooth(gross_const, ROLL_WIN),
        "K_gross": K_gross,
    }


def apply_cap(ax: plt.Axes, y_cap: float | None) -> None:
    """Add a horizontal cap line and optionally shade above it."""
    if y_cap is None:
        return

    ax.axhline(y_cap, color="k", lw=LINE_WIDTH, **CAP_LINE_STYLE)

    ymin, ymax = ax.get_ylim()
    if SHADE_ABOVE_CAP:
        ax.axhspan(y_cap, ymax, color="0.85", alpha=0.5, zorder=0)

    # keep current lower bound, ensure the upper bound is at least y_cap
    ax.set_ylim(ymin, max(y_cap, ymin + 1e-9))


# =============================
# Load & compute
# =============================
v_ref = float(V_REF_USER)
if not np.isfinite(v_ref) or v_ref <= 0:
    raise ValueError("V_REF_USER must be a positive finite number.")

dfP = load_df(raw_pleated)
dfE = load_df(raw_electrostatic)
dfC = load_df(raw_combo)

P = series_from_df(dfP, v_ref=v_ref, n=N_EXPONENT)
E = series_from_df(dfE, v_ref=v_ref, n=N_EXPONENT)
C = series_from_df(dfC, v_ref=v_ref, n=N_EXPONENT)

# Optional: sanity check same time grid (recommended for fair comparisons)
# If you *expect* identical time arrays, enforce it:
if not (np.array_equal(P["t"], E["t"]) and np.array_equal(P["t"], C["t"])):
    raise ValueError(
        "Time arrays differ between datasets. Align/resample them to the same time grid "
        "before using this comparison plot."
    )

t = P["t"]

# =============================
# Plot: 1 row × 3 cols
# =============================
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(14, 4.8), constrained_layout=True)

labels = ["Pleated", "Electrostatic", "Pleated+Electrostatic"]
colors = [C_PLEATED, C_ESTATIC, C_COMBO]
series = [P, E, C]

# Panel 1 — Prefilter ΔP (dp2)
for s, lab, col in zip(series, labels, colors):
    ax0.plot(t, s["dp2"], color=col, lw=LINE_WIDTH, label=lab)
ax0.set_title("Prefilter ΔP — Experimental")
ax0.set_ylabel("Pa")
ax0.set_xlabel("Time (s)")
ax0.grid(True, ls="--", alpha=0.35)
ax0.legend(loc="upper left", frameon=True)

# Panel 2 — Velocity
for s, lab, col in zip(series, labels, colors):
    ax1.plot(t, s["v"], color=col, lw=LINE_WIDTH, label=lab)
ax1.set_title("Velocity — Experimental")
ax1.set_ylabel("m/s")
ax1.set_xlabel("Time (s)")
ax1.grid(True, ls="--", alpha=0.35)
ax1.legend(loc="lower left", frameon=True)

# Panel 3 — Gross ΔP @ constant v_ref
for s, lab, col in zip(series, labels, colors):
    ax2.plot(t, s["gross_const"], color=col, lw=LINE_WIDTH, label=lab)
ax2.set_title("Gross ΔP — Constant Face Velocity")
ax2.set_ylabel("Pa")
ax2.set_xlabel("Time (s)")
ax2.grid(True, ls="--", alpha=0.35)
leg = ax2.legend(loc="lower right", frameon=True, title=f"v_ref = {v_ref:.3f} m/s")
if leg and leg.get_title():
    leg.get_title().set_fontsize(9)
apply_cap(ax2, Y_CAP_GROSS_CONST)

# =============================
# Save figure
# =============================
svg_path = OUTDIR / "prefilter_comparison_constant_velocity.svg"
png_path = OUTDIR / "prefilter_comparison_constant_velocity.png"
fig.savefig(svg_path)
fig.savefig(png_path, dpi=DPI)
plt.close(fig)

print(f"Saved: {svg_path}  |  {png_path}")